In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time
import requests
import logging
import re
import os
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler(f"tesla_job_scraper_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"),
                        logging.StreamHandler()
                    ])
logger = logging.getLogger(__name__)

# Function to scroll and load all jobs
def scroll_to_load_all(driver, max_scrolls=20, wait_time=1.5):
    """
    Scroll the page to load all content with a maximum number of scrolls
    For Tesla career site
    """
    scrolls = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    consecutive_no_change = 0
    
    logger.info("Starting to scroll to load all content...")
    
    while scrolls < max_scrolls:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)  # Wait time for content to load
        
        # Take screenshot for debugging
        driver.save_screenshot(f"screenshots/tesla_scroll_{scrolls+1}.png")
        
        # Try to find "Load More" or similar buttons and click them
        try:
            load_more_buttons = driver.find_elements(By.XPATH, 
                "//button[contains(text(), 'Load More') or contains(text(), 'View More') or contains(text(), 'Show More')]")
            if load_more_buttons:
                for button in load_more_buttons:
                    if button.is_displayed() and button.is_enabled():
                        driver.execute_script("arguments[0].click();", button)
                        logger.info("Clicked 'Load More' button")
                        time.sleep(wait_time + 1)  # Extra wait for new content
        except Exception as e:
            logger.info(f"No 'Load More' button found or error clicking it: {e}")
        
        # Check if we've reached the end of the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        logger.info(f"Scroll {scrolls+1}: Height {last_height} → {new_height}")
        
        if new_height == last_height:
            consecutive_no_change += 1
            logger.info(f"No change detected ({consecutive_no_change}/3)")
            if consecutive_no_change >= 3:  # If no change for 3 consecutive scrolls
                logger.info("No more content loading after multiple scrolls. Stopping scroll operation.")
                break
        else:
            consecutive_no_change = 0
            
        last_height = new_height
        scrolls += 1
    
    logger.info(f"Completed scrolling after {scrolls} scrolls.")
    return True

# Function to handle category navigation on Tesla career page
def handle_job_categories(driver):
    """
    Navigate through job categories on Tesla careers page
    """
    all_jobs = []
    
    logger.info("Finding and processing job categories...")
    
    # Take screenshot before category processing
    driver.save_screenshot("screenshots/tesla_before_categories.png")
    
    # Find job category sections (based on the screenshot, they have card-like elements)
    # The specific selectors might need adjustment based on the actual HTML structure
    try:
        job_categories = driver.find_elements(By.CSS_SELECTOR, ".tcl-section, [id^='tdsgrid-']")
        logger.info(f"Found {len(job_categories)} potential job category sections")
        
        if len(job_categories) == 0:
            # If no specific sections found, we'll try to get all job listings from current page
            logger.info("No specific job categories found. Getting all visible jobs.")
            return extract_job_listings_tesla(driver)
            
        # Process "See Opportunities" links within categories
        opportunity_links = driver.find_elements(By.XPATH, "//a[contains(text(), 'See Opportunities')]")
        logger.info(f"Found {len(opportunity_links)} 'See Opportunities' links")
        
        # If no opportunity links found, try to get jobs from current page
        if len(opportunity_links) == 0:
            logger.info("No 'See Opportunities' links found. Getting jobs from current page.")
            return extract_job_listings_tesla(driver)
        
        # Store the current window handle
        original_window = driver.current_window_handle
        
        # Process each category
        for index, link in enumerate(opportunity_links):
            try:
                # Get category name - usually in a heading element near the link
                category_name = "Unknown Category"
                try:
                    # Try to find the closest heading to this link
                    parent_section = link.find_element(By.XPATH, "./ancestor::section[1]") 
                    headings = parent_section.find_elements(By.XPATH, ".//h2 | .//h3 | .//h4")
                    if headings:
                        category_name = headings[0].text.strip()
                except:
                    # If we can't find a heading, use the link's parent text
                    try:
                        parent_text = link.find_element(By.XPATH, "./..").text.strip()
                        # Extract first line as category name
                        category_name = parent_text.split('\n')[0].strip()
                    except:
                        logger.warning(f"Could not determine category name for link {index+1}")
                
                logger.info(f"Processing category: {category_name} ({index+1}/{len(opportunity_links)})")
                
                # Take screenshot before clicking
                driver.save_screenshot(f"screenshots/tesla_category_{index+1}_before.png")
                
                # Open in new tab to preserve main page navigation
                driver.execute_script("window.open('about:blank', '_blank');")
                driver.switch_to.window(driver.window_handles[-1])
                
                # Get the href value and navigate to it
                href = link.get_attribute('href')
                driver.get(href)
                time.sleep(3)  # Wait for page to load
                
                # Take screenshot after navigation
                driver.save_screenshot(f"screenshots/tesla_category_{index+1}_page.png")
                
                # Scroll to load all jobs in this category
                scroll_to_load_all(driver)
                
                # Extract jobs for this category
                category_jobs = extract_job_listings_tesla(driver)
                
                # Add category to each job
                for job in category_jobs:
                    job['Category'] = category_name
                
                all_jobs.extend(category_jobs)
                logger.info(f"Added {len(category_jobs)} jobs from '{category_name}' category")
                
                # Close tab and switch back to original window
                driver.close()
                driver.switch_to.window(original_window)
                
            except Exception as e:
                logger.error(f"Error processing category {index+1}: {e}")
                # Make sure we're back to the original window
                try:
                    if driver.current_window_handle != original_window:
                        driver.close()
                        driver.switch_to.window(original_window)
                except:
                    pass
                    
        return all_jobs
                
    except Exception as e:
        logger.error(f"Error finding job categories: {e}")
        driver.save_screenshot("screenshots/tesla_categories_error.png")
        
        # Fallback to extracting jobs from current page
        return extract_job_listings_tesla(driver)

# Extract job information from Tesla page
def extract_job_listings_tesla(driver):
    """
    Extract job listings from Tesla career page
    """
    jobs_data = []
    
    # Take screenshot before extraction
    driver.save_screenshot("screenshots/tesla_before_extraction.png")
    
    try:
        # First try to identify if we're on a job listings page with a search form
        search_input = None
        try:
            search_input = driver.find_element(By.XPATH, "//input[@type='search' or @placeholder='Search by role or keyword']")
            logger.info("Found search input on job listings page")
        except:
            logger.info("No search input found. May not be on standard listings page.")
        
        # Look for job listing elements
        # Based on the screenshots, we need to identify the correct job card elements
        job_cards = []
        
        # Try different selectors that might contain job listings
        for selector in [
            "[id^='job-'] a", 
            ".job-card", 
            "a[href*='/careers/'][href*='/apply']",
            "[id^='tdsgrid-'] a[href*='/careers/']",
            ".opening a",  # Similar to Greenhouse job board
            ".positions-list a",
            "section a[href*='/careers/']"
        ]:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                if elements:
                    logger.info(f"Found {len(elements)} potential job elements with selector: {selector}")
                    job_cards.extend(elements)
            except:
                pass
        
        # Remove duplicates by href
        unique_hrefs = set()
        unique_cards = []
        
        for card in job_cards:
            href = card.get_attribute("href")
            if href and href not in unique_hrefs and '/careers/' in href:
                unique_hrefs.add(href)
                unique_cards.append(card)
        
        logger.info(f"Found {len(unique_cards)} unique job listings after deduplication")
        
        if len(unique_cards) > 0:
            # Highlight first job for screenshot
            try:
                driver.execute_script("arguments[0].style.border='3px solid red'", unique_cards[0])
                driver.save_screenshot("screenshots/tesla_job_elements_found.png")
                driver.execute_script("arguments[0].style.border=''", unique_cards[0])
            except:
                pass
        else:
            logger.warning("No job cards found using primary selectors")
            driver.save_screenshot("screenshots/tesla_no_jobs_found.png")
            
            # Try alternative approach - look for any links containing '/careers/' and job-related keywords
            try:
                all_links = driver.find_elements(By.TAG_NAME, "a")
                for link in all_links:
                    href = link.get_attribute("href")
                    if href and '/careers/' in href and href not in unique_hrefs:
                        # Check if link or its parent contains job-related text
                        link_text = link.text.lower()
                        if any(keyword in link_text for keyword in ["engineer", "manager", "specialist", "analyst", "developer", "job", "position", "apply"]):
                            unique_hrefs.add(href)
                            unique_cards.append(link)
                
                logger.info(f"Found {len(unique_cards)} jobs using alternative detection")
            except Exception as e:
                logger.error(f"Error during alternative job detection: {e}")
                
        # Process each job card
        for index, job_card in enumerate(unique_cards):
            try:
                # Get job link
                link = job_card.get_attribute("href")
                
                # Get job title
                title = job_card.text.strip()
                
                # If the text contains multiple lines, the first line is likely the title
                if "\n" in title:
                    title = title.split("\n")[0].strip()
                
                # If title is empty, try to find title element inside
                if not title:
                    try:
                        title_elem = job_card.find_element(By.TAG_NAME, "h3") or job_card.find_element(By.TAG_NAME, "h4")
                        title = title_elem.text.strip()
                    except:
                        # Try to use last part of URL as fallback title
                        if link:
                            url_parts = link.rstrip('/').split('/')
                            title = url_parts[-1].replace('-', ' ').title()
                
                # Get location - often in the job card but might need specific targeting
                location = "Not specified"
                try:
                    # Try various ways to find location text
                    location_elem = None
                    try:
                        # Try to find location by class or common patterns
                        location_elem = job_card.find_element(By.CSS_SELECTOR, 
                            ".location, [class*='location'], span:not(:first-child)")
                    except:
                        pass
                    
                    if not location_elem:
                        # Try to get full text and parse location from it
                        full_text = job_card.text
                        text_parts = full_text.split('\n')
                        if len(text_parts) > 1:
                            location = text_parts[1].strip()
                    else:
                        location = location_elem.text.strip()
                except:
                    pass
                
                # Add job if we have title and link
                if title and title.strip() and link and link.strip():
                    # Check for duplicate before adding
                    is_duplicate = False
                    for existing_job in jobs_data:
                        if existing_job["Title"] == title and existing_job["Link"] == link:
                            is_duplicate = True
                            break
                    
                    if not is_duplicate:
                        jobs_data.append({
                            "Title": title,
                            "Location": location,
                            "Department": "Not specified",  # We'll set this with category later
                            "Link": link,
                            "Description": "",  # We'll get descriptions later
                            "Company": "Tesla"
                        })
                        logger.info(f"Added job: {title} at {location}")
                
            except (StaleElementReferenceException, Exception) as e:
                logger.error(f"Error extracting job details for job {index+1}: {e}")
                continue
                
    except Exception as e:
        logger.error(f"Error finding job elements: {e}")
        driver.save_screenshot("screenshots/tesla_extract_error.png")
    
    return jobs_data

# Function to get job descriptions
def get_job_descriptions(driver, jobs_data, max_descriptions=100):
    """
    Get job descriptions for Tesla jobs by visiting their individual pages
    """
    logger.info(f"Getting descriptions for {len(jobs_data)} jobs (up to {max_descriptions})")
    
    # Create a dedicated folder for job description screenshots
    job_desc_folder = 'tesla_job_description_screenshots'
    os.makedirs(job_desc_folder, exist_ok=True)
    
    # Store current URL to return to afterward
    original_url = driver.current_url
    original_window = driver.current_window_handle
    
    # Process up to max_descriptions
    for i, job in enumerate(jobs_data[:max_descriptions]):
        if not job.get("Link"):
            continue
            
        logger.info(f"Getting description for job {i+1}/{min(len(jobs_data), max_descriptions)}: {job['Title']}")
        
        # Create a clean filename from the job title
        clean_title = re.sub(r'[\\/*?:"<>|]', "", job['Title'])
        clean_title = re.sub(r'\s+', "_", clean_title)
        clean_title = clean_title[:100] if len(clean_title) > 100 else clean_title
        
        # Create a new tab for each job
        try:
            # Open new tab
            driver.execute_script("window.open('about:blank', '_blank');")
            driver.switch_to.window(driver.window_handles[-1])
            
            # Navigate to job details page
            driver.get(job["Link"])
            time.sleep(4)  # Wait for page to load
            
            # Take a screenshot
            screenshot_file = f"{job_desc_folder}/job_{i+1}_{clean_title}.png"
            driver.save_screenshot(screenshot_file)
            
            # Extract description - try various selectors
            description = ""
            desc_selectors = [
                ".job-description", "#job-description", "[id*='job-description']",
                ".description", "#description",
                ".job-details", "#job-details",
                "main", "article", 
                "[role='main']", "[class*='content']"
            ]
            
            for selector in desc_selectors:
                try:
                    desc_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    if desc_elements:
                        description = desc_elements[0].text.strip()
                        if description:
                            logger.info(f"Got description for '{job['Title']}' ({len(description)} chars)")
                            break
                except Exception as e:
                    logger.debug(f"Selector {selector} failed: {e}")
            
            # Update the job with the description
            if description:
                job["Description"] = description
            
            # Close tab
            driver.close()
            driver.switch_to.window(original_window)
            
        except Exception as e:
            logger.error(f"Error getting description for {job['Title']}: {e}")
            # Make sure we're back to the original window
            try:
                driver.close()
                driver.switch_to.window(original_window)
            except:
                pass
    
    # Return to original page
    try:
        driver.get(original_url)
        time.sleep(2)
    except:
        pass
        
    logger.info(f"Completed fetching descriptions for {min(len(jobs_data), max_descriptions)} jobs")
    return jobs_data

# Function to handle different types of popups
def handle_popups(driver):
    try:
        # Common buttons for accepting cookies, terms, etc.
        popup_selectors = [
            "//button[contains(text(), 'Accept')]", 
            "//button[contains(text(), 'I agree')]",
            "//button[contains(@id, 'accept')]",
            "//button[contains(@class, 'accept')]",
            "//button[contains(text(), 'Continue')]",
            "//button[contains(text(), 'Got it')]",
            "//button[contains(text(), 'Close')]",
            "//button[@aria-label='Close']",
            "//div[contains(@class, 'cookie')]//button",
            "//div[contains(@id, 'consent')]//button"
        ]
        
        for xpath in popup_selectors:
            try:
                buttons = driver.find_elements(By.XPATH, xpath)
                for button in buttons:
                    if button.is_displayed():
                        button.click()
                        logger.info(f"Clicked popup/cookie button with xpath: {xpath}")
                        time.sleep(1)
            except Exception:
                continue
                
        # Handle alerts
        try:
            alert = driver.switch_to.alert
            alert.accept()
            logger.info("Accepted alert popup")
        except:
            pass
            
    except Exception as e:
        logger.warning(f"Error handling popups: {e}")

# Function to validate URL
def is_valid_link(url):
    if not url or not isinstance(url, str) or not url.startswith("http"):
        return False
        
    try:
        response = requests.head(url, timeout=5, allow_redirects=True)
        return response.status_code < 400  # Accept any non-error status
    except requests.RequestException:
        logger.warning(f"Invalid link: {url}")
        return False

# Main Tesla job scraper function
def scrape_tesla_jobs(search_keyword="", max_pages=10, headless=False):
    """
    Scrape Tesla jobs from the careers page
    
    Parameters:
    search_keyword (str): Keyword to filter jobs by title
    max_pages (int): Maximum number of pages to scrape (used for pagination if present)
    headless (bool): Whether to run in headless mode
    
    Returns:
    list: List of all job dictionaries found
    """
    options = webdriver.ChromeOptions()
    
    if headless:
        options.add_argument('--headless')
        
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36')
    
    # Create directories for debugging
    os.makedirs('screenshots', exist_ok=True)
    
    driver = None
    jobs_data = []

    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.set_page_load_timeout(30)
        
        # Tesla's careers URL
        base_url = "https://www.tesla.com/careers"
        logger.info(f"Scraping jobs from Tesla" + (f", filtering by keyword '{search_keyword}'" if search_keyword else ""))
        
        # Open the Tesla careers page
        driver.get(base_url)
        driver.save_screenshot("screenshots/tesla_initial.png")
        
        # Handle popups
        handle_popups(driver)
        
        # If there's a search box, use it
        if search_keyword:
            try:
                search_box = driver.find_element(By.XPATH, "//input[@type='search' or @placeholder='Search by role or keyword']")
                if search_box:
                    search_box.clear()
                    search_box.send_keys(search_keyword)
                    search_box.send_keys(Keys.RETURN)
                    time.sleep(3)  # Wait for results to load
                    logger.info(f"Searched for '{search_keyword}'")
                    driver.save_screenshot("screenshots/tesla_search_results.png")
                    
                    # Wait for search results
                    try:
                        # Wait for page to load after search
                        WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.TAG_NAME, "body"))
                        )
                    except TimeoutException:
                        logger.warning("Timed out waiting for search results")
            except NoSuchElementException:
                logger.info("No search box found. Will filter results after scraping.")
        
        # Scroll to load all results on current page
        scroll_to_load_all(driver)
        driver.save_screenshot("screenshots/tesla_after_scroll.png")
        
        # Handle job categories and collect all jobs
        all_jobs = handle_job_categories(driver)
        logger.info(f"Collected {len(all_jobs)} total jobs")
        
        # If search keyword was provided and we didn't use the search box, filter the results by job title
        if search_keyword and all_jobs:
            filtered_jobs = [job for job in all_jobs if search_keyword.lower() in job["Title"].lower()]
            logger.info(f"Filtered from {len(all_jobs)} to {len(filtered_jobs)} jobs matching '{search_keyword}'")
            all_jobs = filtered_jobs
        
        # Get job descriptions for all collected jobs
        jobs_with_descriptions = get_job_descriptions(driver, all_jobs, max_descriptions=100)
        
        # Save all jobs
        jobs_data = jobs_with_descriptions
        logger.info(f"Saving {len(jobs_data)} jobs found in search results")

    except Exception as e:
        logger.error(f"Error during scraping from Tesla: {e}")
        if driver:
            driver.save_screenshot("screenshots/tesla_error.png")

    finally:
        if driver:
            driver.quit()

    return jobs_data

# Main function to scrape and save results to CSV
def main(search_keyword="", max_pages=10, headless=False):
    start_time = time.time()
    logger.info(f"Starting Tesla job scraper at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Create screenshots directory
    os.makedirs('screenshots', exist_ok=True)
    
    # Create job description screenshots directory
    os.makedirs('tesla_job_description_screenshots', exist_ok=True)
    
    # Scrape Tesla jobs
    jobs_data = scrape_tesla_jobs(search_keyword, max_pages, headless)
    
    # Generate filenames with timestamps
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    keyword_slug = search_keyword.replace(' ', '_') if search_keyword else 'all'
    detailed_filename = f"tesla_jobs_detailed_{keyword_slug}_{timestamp}.csv"
    simple_filename = f"tesla_jobs_simple_{keyword_slug}_{timestamp}.csv"
    
    # Save results
    if jobs_data:
        # Create DataFrame
        df = pd.DataFrame(jobs_data)
        
        # Save detailed CSV with descriptions
        df.to_csv(detailed_filename, index=False, encoding='utf-8-sig')
        logger.info(f"Detailed jobs saved to '{detailed_filename}'")
        
        # Create and save a simplified CSV without descriptions
        simple_df = df[['Title', 'Location', 'Department', 'Link']].copy()
        simple_df.to_csv(simple_filename, index=False, encoding='utf-8-sig')
        logger.info(f"Simplified jobs list saved to '{simple_filename}'")
        
        # Print summary
        print("\n" + "="*50)
        print(f"Tesla Job Scraping Results:")
        print(f"Total jobs found: {len(df)}")
        print(f"Unique locations: {len(df['Location'].unique())}")
        print(f"Sample jobs:")
        print(df[['Title', 'Location']].head())
        print("\nTop locations:")
        print(df['Location'].value_counts().head())
        print(f"\nResults saved to:")
        print(f"- {detailed_filename}")
        print(f"- {simple_filename}")
        print("="*50)
        
        elapsed_time = time.time() - start_time
        logger.info(f"Completed in {elapsed_time:.2f} seconds")
        
        return df
    else:
        logger.warning(f"No jobs found with search keyword '{search_keyword}'")
        print("\nNo jobs found to display.")
        
        elapsed_time = time.time() - start_time
        logger.info(f"Process completed with no results in {elapsed_time:.2f} seconds")
        
        return pd.DataFrame()

if __name__ == "__main__":
    print("Tesla Job Scraper")
    print("="*50)
    
    # Get user input
    job_title = input("Enter job title to search for (leave empty to get all jobs): ").strip()
    
    try:
        max_pages = int(input("Maximum number of pages to scrape (default 10): ") or "10")
    except ValueError:
        max_pages = 10
        print("Invalid input. Using default of 10 pages.")
    
    headless_mode = input("Run in headless mode? (y/n, default: n): ").strip().lower() == 'y'
    
    print("\nStarting job scraper...")
    print("This may take several minutes depending on the number of jobs and pages.")
    print("Progress will be logged to the console and a log file.")
    
    # Run the scraper
    main(job_title, max_pages, headless_mode)

Tesla Job Scraper


Enter job title to search for (leave empty to get all jobs):  Machine Learning
Maximum number of pages to scrape (default 10):  1
Run in headless mode? (y/n, default: n):  y


2025-03-17 23:54:55,047 - INFO - Starting Tesla job scraper at 2025-03-17 23:54:55
2025-03-17 23:54:55,050 - INFO - ====== WebDriver manager ======



Starting job scraper...
This may take several minutes depending on the number of jobs and pages.
Progress will be logged to the console and a log file.


2025-03-17 23:54:55,341 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-17 23:54:55,403 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-17 23:54:55,449 - INFO - Driver [/Users/srikar/.wdm/drivers/chromedriver/mac64/134.0.6998.88/chromedriver-mac-arm64/chromedriver] found in cache
2025-03-17 23:54:56,481 - INFO - Scraping jobs from Tesla, filtering by keyword 'Machine Learning'
2025-03-17 23:55:06,790 - INFO - Searched for 'Machine Learning'
2025-03-17 23:55:07,782 - INFO - Starting to scroll to load all content...
2025-03-17 23:55:09,883 - INFO - Scroll 1: Height 2180 → 2180
2025-03-17 23:55:09,883 - INFO - No change detected (1/3)
2025-03-17 23:55:12,014 - INFO - Scroll 2: Height 2180 → 2180
2025-03-17 23:55:12,014 - INFO - No change detected (2/3)
2025-03-17 23:55:14,213 - INFO - Scroll 3: Height 2180 → 2180
2025-03-17 23:55:14,213 - INFO - No change detected (3/3)
2025-03-17 23:55:14,214 - INFO - No more content loading after multiple scrolls. S


Tesla Job Scraping Results:
Total jobs found: 13
Unique locations: 1
Sample jobs:
                                               Title       Location
0  Machine Learning Engineer, Motion Planning, Se...  Not specified
1  Machine Learning Engineer, Geometric Vision, S...  Not specified
2  Sr. Machine Learning Engineer, Navigation, Opt...  Not specified
3  Machine Learning Engineer, Reliability Enginee...  Not specified
4  Machine Learning Hardware Performance Engineer...  Not specified

Top locations:
Location
Not specified    13
Name: count, dtype: int64

Results saved to:
- tesla_jobs_detailed_Machine_Learning_20250317_235630.csv
- tesla_jobs_simple_Machine_Learning_20250317_235630.csv
